<a href="https://colab.research.google.com/github/hukim1112/MLDL/blob/master/object_detection/Inferencing_with_faster_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pycocotools : library to handle your coco style json
#!pip install pycocotools
# colab에는 pycocotools가 이미 설치되어 있음.

# 1. 구글 드라이브에서 MS COCO 데이터셋 마운트해 로컬 드라이브로 복사해오기

In [ ]:
!cp /content/drive/MyDrive/4.teaching/3.롯데정보통신/데이터/coco/val2017.zip ./val2017.zip
!cp /content/drive/MyDrive/4.teaching/3.롯데정보통신/데이터/coco/instances_val2017.json ./instances_val2017.json

In [ ]:
!unzip /content/val2017.zip #압축해제

# 2. 이미지 준비하기

In [ ]:
from pycocotools.coco import COCO
import os
import cv2
from matplotlib import pyplot as plt

In [ ]:
coco=COCO("/content/instances_val2017.json") #create COCO object from your .json

ids = coco.getImgIds()
id = ids[0]

In [ ]:
label_set = ["person","bicycle","car","motorcycle","airplane","bus","train","truck","boat","traffic light","fire hydrant","street sign","stop sign","parking meter","bench","bird","cat","dog","horse","sheep","cow","elephant","bear","zebra","giraffe","hat","backpack","umbrella","shoe","eye glasses","handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite","baseball bat","baseball glove","skateboard","surfboard","tennis racket","bottle","plate","wine glass","cup","fork","knife","spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot","hot dog","pizza","donut","cake","chair","couch","potted plant","bed","mirror","dining table","window","desk","toilet","door","tv","laptop","mouse","remote","keyboard","cell phone","microwave","oven","toaster","sink","refrigerator","blender","book","clock","vase","scissors","teddy bear","hair drier","toothbrush","hair brush"]

In [ ]:
print(len(label_set))

In [ ]:
coco.loadImgs(id) #id로 이미지 정보 가져오기.

In [ ]:
image_dir = "/content/val2017"

def get_image(image_id, coco):
  image_info = coco.loadImgs(image_id)[0]
  filename = image_info['file_name']
  original_size = (int(image_info['height']), int(image_info['width']))
  image_path = os.path.join(image_dir, filename)
  image = cv2.imread(image_path)
  if image is None:
    filename = os.path.splitext(filename)[0] + '.JPG'
    image_path = os.path.join(image_dir, filename)
    image = cv2.imread(image_path)
    if image is None:
      raise FileNotFoundError("Can't not find correct filepath", filename)
      print(filename)
  image = image[:,:,::-1]
  return image

image = get_image(ids[0], coco)

In [ ]:
plt.imshow(image)
plt.show()

# 3. Load faster rcnn model

In [ ]:
import tensorflow_hub as hub

# Apply image detector on a single image.
detector = hub.load("https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_640x640/1")

In [ ]:
image_tensor = tf.expand_dims(image, axis=0)
print(image_tensor.shape)

# 4. Faster RCNN의 결과값.


In [ ]:
detector_output = detector(image_tensor)

Outputs
The output dictionary contains:

- num_detections: a tf.int tensor with only one value, the number of detections [N].
- detection_boxes: a tf.float32 tensor of shape [N, 4] containing bounding box coordinates in the following order: [ymin, xmin, ymax, xmax].
- detection_classes: a tf.int tensor of shape [N] containing - - detection class index from the label file.
- detection_scores: a tf.float32 tensor of shape [N] containing detection scores.
- raw_detection_boxes: a tf.float32 tensor of shape [1, M, 4] containing decoded detection boxes without Non-Max suppression. M is the number of raw detections.
- raw_detection_scores: a tf.float32 tensor of shape [1, M, 90] and contains class score logits for raw detection boxes. M is the number of raw detections.
- detection_anchor_indices: a tf.float32 tensor of shape [N] and contains the anchor indices of the detections after NMS.
- detection_multiclass_scores: a tf.float32 tensor of shape [1, N, 90] and contains class score distribution (including background) for detection boxes in the image including background class.

In [ ]:
detector_output["num_detections"] # 300개의 positive anchor를 사용(Non-Maximum Suppression 이후)

In [ ]:
detector_output["detection_boxes"] # 300개의 positive anchor들에 대한 수정좌표

In [ ]:
detector_output["detection_classes"] # 300개의 positive anchor들의 클래스 예측

In [ ]:
detector_output["detection_scores"] # 300개의 positive anchor들에 대한 확신도(confidence score)

In [ ]:
detector_output['detection_multiclass_scores'] # 300개의 positive anchor들에 대한 클래스 확률 예측

In [ ]:
tf.reduce_sum(detector_output['detection_multiclass_scores'], axis=-1) #softmax 확률 분포인 것 확인

# 5. 결과값 시각화

In [ ]:
confidence = detector_output["detection_scores"]
true_mask = confidence>0.5
true_scores = confidence[true_mask].numpy()
true_labels = detector_output["detection_classes"][true_mask].numpy().astype(np.int32)
true_boxes = detector_output["detection_boxes"][true_mask].numpy()

In [ ]:
print("num boxes :", len(true_boxes), true_boxes)
print("num labels :", len(true_labels), true_labels)

In [ ]:
a = np.array([[10, 20, 30, 40, 50],
              [6,  7,  8,  9,  10]])
print(a[:, [0, 2, 4, 3, 1]])

In [ ]:
true_boxes = true_boxes[:,[1,0,3,2]].copy() #y1,x1,y2,x2 => x1,y1,x2,y2

In [ ]:
import cv2
import numpy as np

class Visualizer():
    def __init__(self, label_set, show_score=True, class_color=None):
        self.label_set = label_set
        self.show_score = show_score
        self.font_args = {'font':cv2.FONT_HERSHEY_SIMPLEX, 'font_scale':1,
                    'font_color':(0,200,200), 'line_type':2,
                    'box_thickness':2, 'box_color':(0, 0, 255),
                    'pred_box_color' :(0, 0, 255)}
        self.class_color = class_color
    def draw_label_and_box(self, img, label, box, visual_arg):
        cv2.rectangle(img, tuple( (int(box[0]), int(box[1])) ), tuple( (int(box[2]), int(box[3])) ),
                visual_arg['box_color'], visual_arg['box_thickness'] )
        cv2.putText(img, label, tuple( (int(box[0]), int(box[3])) ),
                visual_arg['font'], visual_arg['font_scale'],
                visual_arg['font_color'], visual_arg['line_type'])
        return img
    def visualize_one_image(self, img, boxes, classes, scores):
        height, width = img.shape[0], img.shape[1]
        visual_arg = self.font_args.copy()
        if len(boxes) != 0:
            detected = True
        else:
            detected = False
        for _box, _class, _score in zip(boxes, classes, scores):
            _label = self.label_set[_class-1]
            if self.show_score==True:
                template = "{} : {:2.2}".format(_label, _score)
            else:
                template = "{}".format(_label)
            if self.class_color is not None:
                visual_arg['box_color'] = self.class_color[_class-1]
                visual_arg['font_color'] = self.class_color[_class-1]
            _box = _box * np.array([width, height, width, height])
            img = self.draw_label_and_box(img, template, _box, visual_arg)
        return img, detected

    def __call__(self, _input, boxes, classes, scores):
        _input = np.array(_input, np.float32)
        if len(_input.shape) == 3:
            output = self.visualize_one_image(_input, boxes, classes, scores)
        else:
            raise ValueError("Wrong input shape : {}".format(_input.shape))
        return output

In [ ]:
visualizer = Visualizer(label_set, show_score=False)

output, detected = visualizer(image, true_boxes, true_labels, true_scores)
print(output.shape, detected)

In [ ]:
plt.imshow(output/255.)

In [ ]:
def inference(image):
  image_tensor = tf.expand_dims(image, axis=0)
  detector_output = detector(image_tensor)
  confidence = detector_output["detection_scores"]
  true_mask = confidence>0.5
  true_scores = confidence[true_mask].numpy()
  true_labels = detector_output["detection_classes"][true_mask].numpy().astype(np.int32)
  true_boxes = detector_output["detection_boxes"][true_mask].numpy()
  true_boxes = true_boxes[:,[1,0,3,2]].copy()
  output, detected = visualizer(image, true_boxes, true_labels, true_scores)
  if detected:
    plt.imshow(output/255.)
    plt.show()

In [ ]:
inference(image)

In [ ]:
ids = coco.getImgIds()
for _id in ids[:5]:
  image = get_image(_id, coco)
  inference(image)